Stanislas Deneuville - Emmanuel Ferrandi - Pol Grisart - Marine Médard
# Project of data science :  Face recognition in a video and counting
12/11/2018

## Part I : Cleaning

In [ ]:
import cv2
import sys
import os
import numpy as np

__face_detection__ : dectect the faces in a photograph.


In : 
* imagePath : name of the file containing the image in the format jpeg for example

Out : 
* face_locations : list of the location of all faces in the picture in parameter
* height : height of the input image
* width : width of the input image


In [ ]:
def detectFaces(image, save=True):
    cascPath = "haarcascade_frontalface_default.xml"

    # Create the haar cascade
    faceCascade = cv2.CascadeClassifier(cascPath)

    # Read the image
    if (save) :
        image = cv2.imread(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    height = image.shape[0]
    width = image.shape[1] 

    # Detect faces in the image
    face_locations = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=3,
        minSize=(20, 20),
        flags = cv2.CASCADE_SCALE_IMAGE
    )

    return face_locations,height,width

__checkRecovery__ : check if there are some recoveries coordonates given and a list of rectangle

In : 
* x : x-coordonate of the upper left corner of the white rectangle
* y : y-coordonate of the upper left corner of the white rectangle
* w : width of the white rectangle
* h : height of the white rectangle
* list_rectangles : list containing the locations of the white rectangles 

Out : 
* True : if there is at least one recovery detected
* False : if there are no recoveries detected

In [ ]:
def checkRecovery(x,y,w,h,list_rectangles):
    if (len(list_rectangles)>1):
        for (x1,y1,w1,h1) in list_rectangles:
            x_set=np.arange(x,x+w)
            y_set=np.arange(y,y+h)
            x1_set=np.arange(x1,x1+w1)
            y1_set=np.arange(y1,y1+h1)
            
            #if there is at least one common value in x-axis and y-axis between the 2 rectangles
            if (bool(set(x_set)&set(x1_set)) & bool(set(y_set)&set(y1_set))):
                return True #there is recovery
        return False
    
    #if there is only one rectangle in the list
    else :
        x_set=np.arange(x,x+w)
        y_set=np.arange(y,y+h)
        x1_set=np.arange(list_rectangles[0][0],list_rectangles[0][0]+list_rectangles[0][2])
        y1_set=np.arange(list_rectangles[0][1],list_rectangles[0][1]+list_rectangles[0][3])
        if (bool(set(x_set)&set(x1_set)) & bool(set(y_set)&set(y1_set))):
            return True
        return False
        

__removeRecoveries__ : remove recovered rectangles in the list of faces
In : 
* face_locations : list containing the locations of the white rectangles 

Out : 
* new_face_locations : list containing the new locations of the white rectangles 

In [ ]:
def removeRecoveries(face_locations):
    new_face_locations = []
    new_face_locations.append(face_locations[0]) #face_locations always has more than one face in this case
    for (x, y, w, h) in face_locations[1:]:
        
        #if next rectangles have no recovery with all the other rectangles
        if not (checkRecovery(x,y,w,h,new_face_locations)):
            #we can keep it in the image
            new_face_locations.append([x,y,w,h])
            
    return new_face_locations

__image_generation__ : generate a black image with white circle whoes location in gave in parameter


In : 
* face_locations : list containing the locations of the white rectangles that will be added to the picture
* height : height of the input image
* width : width of the input image

Out : 
* BWimage : list of list  representing a black picture with white rectangles


In [ ]:
def toBlackAndWhite(face_locations,height,width):
    BWimage = np.zeros((height,width))
    height = BWimage.shape[0]
    
    for (x, y, w, h) in face_locations:
        for i in range(h):
            for j in range(w):
                BWimage[y+j,x+i]=255 #255 is black
                
    return BWimage

__resize_image__ : resize the image gave in parameter in a 50x50 pixels image 


In : 
* image : list of list  representing an image in white and black

Out : 
* face_locations : list of list representing the same image than in parameter but with the dimensions 50x50


In [ ]:
def resize_image(image) :
    final_image = cv2.resize(image, (50,50), interpolation = cv2.INTER_AREA) 
    return final_image

__save_image__ : save the image in parameter in the good repository according to nb_circles


In : 
* image : image that we want to save
* nb_faces : int, the number of faces detected in the image
* cpt : int, the numero of the image that is being cleaned

Out : 
* int : 0 if everything is ok


In [ ]:
def save_image(image, nb_faces, cpt) :
    path = "./train_set/"+str(nb_faces)+"/"
    if (not os.path.isdir(path)) :
        os.mkdir(path)
    cv2.imwrite(path+"image"+str(cpt)+".jpg",image)
    #print("Image saved in : "+path+"image"+str(cpt)+".jpg")
    return 0

__cleaning__ : clean an image so that we can count faces in it thanks to an ANN

In : 
* image : name of the file containing the image in the format jpeg for example
* cpt : the numero of the image that is being cleaned

Out :
* final_image : list of lists, the final image


In [ ]:
def cleaning(image, cpt) :
    face_locations,height,width = detectFaces(image)
    nb_faces = len(face_locations)
    
    if(nb_faces>1):
        face_locations = removeRecoveries(face_locations)
        nb_faces = len(face_locations)
    
    newImage = toBlackAndWhite(face_locations,height,width)
    
    final_image = resize_image(newImage)
    
    save_image(final_image, nb_faces, cpt)
    
    return final_image
    
cleaning('test1.jpg',0)

__clean_without_save__ : it is the same function as cleaning but it does not save the new image in repository

In : 
* image : image that need to be cleaned

Out :
* final_image : list of list, the cleaned image


In [ ]:
def clean_without_save(image) :
    face_locations,height,width = detectFaces(image, False)
    nb_faces = len(face_locations)
    if(nb_faces>1):
        face_locations = removeRecoveries(face_locations)
        nb_faces = len(face_locations)
    
    newImage = toBlackAndWhite(face_locations,height,width)
    
    final_image = resize_image(newImage)
    
    return final_image
    

__cleaAll__ : clean all images in a directory

In : 
* name_dir : string, name of the directory where we want to clean all the images



In [ ]:
def cleanAll(name_dir) :
    cpt = 0
    liste_files = os.listdir(name_dir)
    for name_f in liste_files :
        if ('.jpg' in name_f) :
            cleaning(name_dir + '/' + name_f, cpt)
            cpt +=1
        
#cleanAll('Group2a')

